In [1]:
import os
#import sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import scipy
#import code

#import tensorflow.python.platform

#import pandas as pd
import numpy as np
import tensorflow as tf
import math


# Dir for train
data_dir = "../data/training/"
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 


# Dir for test
root_dir = "../data/test_set_images/"

TRAINING_SIZE = 100
TEST_SIZE = 50

C:\Users\Brice\Anaconda3\envs\myenvironment\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [3]:
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are scaled from [0, 1] instead of [0,255].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    
    return np.asarray(imgs)

train_raw = extract_data(train_data_filename, TRAINING_SIZE)
ground_truth_raw = extract_data(train_labels_filename, TRAINING_SIZE)

Loading ../data/training/images/satImage_001.png
Loading ../data/training/images/satImage_002.png
Loading ../data/training/images/satImage_003.png
Loading ../data/training/images/satImage_004.png
Loading ../data/training/images/satImage_005.png
Loading ../data/training/images/satImage_006.png
Loading ../data/training/images/satImage_007.png
Loading ../data/training/images/satImage_008.png
Loading ../data/training/images/satImage_009.png
Loading ../data/training/images/satImage_010.png
Loading ../data/training/images/satImage_011.png
Loading ../data/training/images/satImage_012.png
Loading ../data/training/images/satImage_013.png
Loading ../data/training/images/satImage_014.png
Loading ../data/training/images/satImage_015.png
Loading ../data/training/images/satImage_016.png
Loading ../data/training/images/satImage_017.png
Loading ../data/training/images/satImage_018.png
Loading ../data/training/images/satImage_019.png
Loading ../data/training/images/satImage_020.png
Loading ../data/trai

Loading ../data/training/groundtruth/satImage_096.png
Loading ../data/training/groundtruth/satImage_097.png
Loading ../data/training/groundtruth/satImage_098.png
Loading ../data/training/groundtruth/satImage_099.png
Loading ../data/training/groundtruth/satImage_100.png


In [4]:
### Extract_test

def extract_test(image_filename):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are scaled from [0, 1] instead of [0,255].
    """
    if os.path.isfile(image_filename):
        print ('Loading ' + image_filename)
        return mpimg.imread(image_filename)
    
# Get filenames and images for all the 50 submission images
image_dir = [root_dir + "test_{}/".format(i) for i in range(1, TEST_SIZE+1)]
filenames = [fn for imdir in image_dir for fn in os.listdir(imdir)]
images = np.asarray([extract_test(image_dir[i-1] + filenames[i-1]) for i in range(1, TEST_SIZE+1)])


Loading ../data/test_set_images/test_1/test_1.png
Loading ../data/test_set_images/test_2/test_2.png
Loading ../data/test_set_images/test_3/test_3.png
Loading ../data/test_set_images/test_4/test_4.png
Loading ../data/test_set_images/test_5/test_5.png
Loading ../data/test_set_images/test_6/test_6.png
Loading ../data/test_set_images/test_7/test_7.png
Loading ../data/test_set_images/test_8/test_8.png
Loading ../data/test_set_images/test_9/test_9.png
Loading ../data/test_set_images/test_10/test_10.png
Loading ../data/test_set_images/test_11/test_11.png
Loading ../data/test_set_images/test_12/test_12.png
Loading ../data/test_set_images/test_13/test_13.png
Loading ../data/test_set_images/test_14/test_14.png
Loading ../data/test_set_images/test_15/test_15.png
Loading ../data/test_set_images/test_16/test_16.png
Loading ../data/test_set_images/test_17/test_17.png
Loading ../data/test_set_images/test_18/test_18.png
Loading ../data/test_set_images/test_19/test_19.png
Loading ../data/test_set_image

In [5]:
def process(train, ground_truth=None, gt=True):
    X_train = (train-0.5)/0.5
    if gt:
        Y_ground_truth = (ground_truth - ground_truth.min())/(ground_truth.max()-ground_truth.min())
        return X_train, Y_ground_truth
    else:
        return X_train

In [6]:
train, ground_truth = process(train_raw, ground_truth_raw)
#test_images = process(images, gt=False)

In [7]:
def data_augmentation(train, ground_truth):
    imgs_processed=[]
    labels_processed = []
    
    for i in range(len(train)):
        
        img = train[i]
        img_gt = ground_truth[i]
        
        img_ud = np.flipud(img)
        img_gt_ud = np.flipud(img_gt)

        img_lr = np.fliplr(img)
        img_gt_lr = np.fliplr(img_gt)

        img_180 = np.rot90(img, 1)
        img_gt_180 = np.rot90(img_gt, 1)

        angle= np.random.randint(180)
        img_angle1 = scipy.ndimage.interpolation.rotate(img, angle, mode='reflect', reshape=False, order=0)
        img_gt_angle1 = scipy.ndimage.interpolation.rotate(img_gt, angle, mode='reflect', reshape=False, order=0)

        angle= np.random.randint(180)
        img_angle2 = scipy.ndimage.interpolation.rotate(img, angle, mode='reflect', reshape=False, order=0)
        img_gt_angle2 = scipy.ndimage.interpolation.rotate(img_gt, angle, mode='reflect', reshape=False, order=0)

        angle= np.random.randint(180)
        img_angle3 = scipy.ndimage.interpolation.rotate(img, angle, mode='reflect', reshape=False, order=0)
        img_gt_angle3 = scipy.ndimage.interpolation.rotate(img_gt, angle, mode='reflect', reshape=False, order=0)

        
        imgs_processed.extend([img, img_ud, img_lr, img_180, img_angle1, img_angle2, img_angle3])
        labels_processed.extend([img_gt, img_gt_ud, img_gt_lr, img_gt_180, img_gt_angle1, img_gt_angle2, img_gt_angle3])
    
    return imgs_processed, labels_processed

In [8]:
augmented_train, augmented_ground_truth = data_augmentation(train, ground_truth)

# Padding part and crop

In [9]:
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    for i in range(padding,imgheight+padding,stride):
        for j in range(padding,imgwidth+padding,stride):
            im_patch = im[j-padding:j+w+padding, i-padding:i+h+padding, :]
            list_patches.append(im_patch)
    return list_patches

In [10]:
def crop_and_padding(train, ground_truth, patch_size, stride, window_size):
    padding = (window_size - patch_size)//2
    
    crop_and_padding_train = []
    crop_and_padding_ground_truth = []
    
    for i in range(len(train)):
        cp_train = img_crop(train[i], patch_size, patch_size, stride, padding)
        cp_gt = img_crop_gt(ground_truth[i], patch_size, patch_size, stride)
        
        crop_and_padding_train.extend(cp_train)
        crop_and_padding_ground_truth.extend(cp_gt)
    
    return crop_and_padding_train, crop_and_padding_ground_truth

In [11]:
#train_data, train_gt = crop_and_padding(augmented_train, augmented_ground_truth, 16, 16, 40)

## Create balanced sets

In [12]:

def balanced_data(train_data, train_gt, threshold=0.25):
    
    labels = np.array([(np.mean(train_gt[i]) > threshold)*1 for i in range(len(train_gt))])
    idx0 = [i for i,v in enumerate(labels) if v==0]
    idx1 = [i for i,v in enumerate(labels) if v==1]
    min_c = min(len(idx0),len(idx1))
    print(2*min_c)
    new_indices = idx0[0:min_c] + idx1[0:min_c]
    train_data = [train_data[i] for i in new_indices]
    train_gt = [train_gt[i] for i in new_indices]
    labels = [(np.mean(train_gt[i]) > threshold)*1 for i in range(len(train_gt))]
    return train_data, train_gt, labels
    

In [13]:
#t,_, labels = balanced_data(train_data, train_gt)

# Utils for model

In [14]:
def conv2d(layer, filters_size, kernel_size, s, padding, activation='relu'):
    if activation=='relu':
        activation=tf.nn.relu
    return tf.layers.conv2d(layer, filters=filters_size,  kernel_size=[kernel_size, kernel_size], strides=[s, s], padding=padding, activation=activation)

def pooling(layer, k=2, s=2, pool_type='max'):
    
    if pool_type=='max':
        return tf.layers.max_pooling2d(layer, pool_size=[k,k], strides=s)
    
def dense(layer, inputs_size, outputs_size, he_std=0.1):
    weights = tf.Variable(tf.truncated_normal([inputs_size, outputs_size],stddev=he_std))
    biases = tf.Variable(tf.constant(he_std, shape=[outputs_size]))
    layer = tf.matmul(layer,weights) + biases  
    return layer

def flattening_layer(layer):
        #make it single dimensional
        input_size = layer.get_shape().as_list()
        new_size = input_size[-1] * input_size[-2] * input_size[-3]
        return tf.reshape(layer, [-1, new_size]),new_size
    

def activation(layer, activation='no_activation'):
    if activation=='relu':
        return tf.nn.relu(layer)
    elif activation=='soft_max':
        return tf.nn.softmax(layer)
    elif activation=='sigmoid':
        return tf.nn.sigmoid(layer)
    else:
        return layer
    
def optimizer_choice(name='GD', lr=0.003):
    if name=='GD':
        return tf.train.GradientDescentOptimizer(lr)
    elif name=='adam':
        return tf.train.AdamOptimizer(lr)

# Train model

# Change in prepare_batch the 480 by n

In [156]:
class CNN:
    def __init__(self, window_size=72, patch_size=16, stride=16, p=0.25):
        print('init done')
        self.X = None
        self.y = None
        self.p = 0.25
        self.network = None
        self.training_op = None
        self.out = None
        self.session = None
        self.stride = stride
        self.window_size=window_size
        self.patch_size=patch_size
        self.padding=(self.window_size-self.patch_size)//2
        self.accuracy = None
        self.loss = None
    
        
        
    def model(self):
        # init the graph 
        #graph = tf.Graph()
        #graph.seed = 1

        tf.reset_default_graph()

        #with graph.as_default():

        # init the placeholder (None is given for more flexibility in batch_size)
        # 16,16 will be put as parameters in a further version
        self.X = tf.placeholder(
                tf.float32,
                shape=[None, self.window_size, self.window_size, 3], name='X')

        self.y = tf.placeholder(tf.float32, shape=[None, 1], name='y')
        self.p = tf.placeholder(tf.float32, name='p')

         # Initialize first convolution step 
        self.network = conv2d(self.X, 64, 5, 1, 'SAME', tf.nn.relu)
        self.network = pooling(self.network)
        self.network = tf.nn.dropout(self.network, self.p)

        self.network = conv2d(self.network, 128, 3, 1, 'SAME', tf.nn.relu)
        self.network = pooling(self.network)
        self.network = tf.nn.dropout(self.network, self.p)

        self.network = conv2d(self.network, 256, 3, 1, 'SAME', tf.nn.relu)
        self.network = pooling(self.network)
        self.network = tf.nn.dropout(self.network, self.p)

        self.network = conv2d(self.network, 256, 3, 1, 'SAME', tf.nn.relu)
        self.network = pooling(self.network)
        self.network = tf.nn.dropout(self.network, self.p)


        # flatten last convolution step for full connected NN
        self.network, flatten_size = flattening_layer(self.network)

        # Initialize first full connected step
        self.network = dense(self.network, 
                     flatten_size, 
                     1)
        #self.network = tf.nn.dropout(self.network, self.p*2, name='out')
        self.network = tf.nn.dropout(self.network, self.p)

        self.out = self.network
        self.out = tf.identity(self.out, name='out')
        
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.y, logits=self.out)
        self.loss = tf.reduce_mean(cross_entropy, name='loss')


        ## Initialize the choosen Optimizer
        optimizer = optimizer_choice('adam', 0.001)
        self.training_op = optimizer.minimize(self.loss, name='training_op')


        # Initialize all Variables
        #init = tf.global_variables_initializer()

        predicted = tf.nn.sigmoid(self.out)
        correct_pred = tf.equal(tf.round(predicted), self.y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
 
    def prepare_batches(self, seq, step):
        n = len(seq)
        print(n)
        res = []
        for i in range(0, n, step):
            res.append(seq[i:i+step])
        return res
    
    
    def train(self, train, ground_truth, batchsize=120, number_epochs=1, saver_filename=None, to_save=True):
        assert saver_filename != None
        cost_history = np.empty(shape=[1],dtype=float)
                
        train1, ground_truth1 = crop_and_padding(train, ground_truth, self.patch_size, self.stride, self.window_size)
        # Balanced data to improve performance
        train_data, gt, labels = balanced_data(train1, ground_truth1)
        
        print(train_data[0].min(), train_data[2].max(), train_data[0].mean())
        print(np.array(labels).mean())
        
        #np.random.seed(999)
        
        with tf.Session() as sess:
            self.session = sess
            self.session.run(tf.global_variables_initializer())


            for iepoch in range(number_epochs):
                
                print(iepoch)
                train_idx_shuffle = np.arange(len(train_data))
                np.random.shuffle(train_idx_shuffle)
                print(train_idx_shuffle)
                batches = self.prepare_batches(train_idx_shuffle, batchsize)        

                for nb, idx in enumerate(batches):
                        

                    #gt_batch = np.array(gt)[idx]
                    #X_batch = tf.convert_to_tensor([train_data[j] for j in idx], dtype=tf.float32)
                    #Y_batch = tf.convert_to_tensor([labels[j] for j in idx], dtype=tf.float32)#np.array(labels)[idx].reshape(-1,1)
                    
                    gt_batch = [gt[j] for j in idx]
                    X_batch = [train_data[j] for j in idx]
                    Y_batch = np.array([labels[j] for j in idx]).reshape(-1,1)

                    feed_dict = {
                            self.X: X_batch,
                            self.y: Y_batch,
                            self.p: 0.25
                           }
                    self.session.run(self.training_op, feed_dict=feed_dict)
                    cost, _, acc = self.session.run([self.loss, self.training_op, self.accuracy], feed_dict={
                                         self.X: X_batch, self.y: Y_batch, self.p: 1})
                    cost_history = np.append(cost_history, acc)

                    if nb % 100 == 0:
                        print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                            nb, cost, acc))
            if to_save:
                saver = tf.train.Saver() 
                saver.save(self.session, '../models/' + saver_filename + '.ckpt')
            
            return X_batch, Y_batch, gt_batch
            
    def predict(self, images, to_submit=False, to_submit_filename=None, restore=False, saver_filename=None):
        
        if restore:
            sess = tf.Session()
            saver = tf.train.Saver()
            saver.restore(sess, '../models/' + saver_filename + '.ckpt')
        else:
            print('Restoring is needed to predict.')
            
        if to_submit:
            with open(to_submit_filename, 'w') as f:
                f.write('id,prediction\n')
                for test in range(1, TEST_SIZE+1,1):
                    print(test)
                    f.writelines('{}\n'.format(s) for s in self.submission_per_patch(sess, images, test))
        else:
            test_patches = np.array(img_crop(images, self.patch_size, self.patch_size, self.stride, self.padding))
            Z = sess.run(self.out, feed_dict={self.X: test_patches, self.p: 1})
            prediction = Z.reshape(-1)
            prediction = np.array([sigmoid(i) for i in prediction])
            prediction = (prediction > 0.5) *1
            
            return Z, prediction
        
            
    def submission_per_patch(self, session, images, img_number):    

        test_patches = np.array(img_crop(images[img_number-1], self.patch_size, self.patch_size, self.stride, self.padding))
        Z = session.run(self.out, feed_dict={self.X: test_patches, self.p: 1})
        prediction = Z.reshape(-1)
        prediction = np.array([sigmoid(i) for i in prediction])
        prediction = (prediction > 0.5) *1
        if img_number == 2:
            print(prediction)
            print(int(prediction[8]))
        patch_size = 16
        nb = 0
        print("Processing " + str(img_number-1))
        for j in range(0, images[img_number-1].shape[1], patch_size):
            for i in range(0, images[img_number-1].shape[0], patch_size):
                label = int(prediction[nb])
                nb += 1
                yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


In [157]:
cnn_TEST = CNN(window_size=62)
cnn_TEST.model()

init done


# Change number_epochs to 20 and saver_filename

In [144]:
x_TEST, y_TEST, gt_TEST = cnn_TEST.train(augmented_train, augmented_ground_truth, saver_filename='model_10_epochs_72_window_size', number_epochs=10)

227776
-1.0 0.64705884 -0.6153779
0.5
0
[105005 175619 195220 ...    393 109352  51053]
227776
Step:     0	Loss: 0.733	Acc: 55.00%
Step:   100	Loss: 0.685	Acc: 59.17%
Step:   200	Loss: 0.680	Acc: 57.50%
Step:   300	Loss: 0.694	Acc: 60.00%
Step:   400	Loss: 0.644	Acc: 62.50%
Step:   500	Loss: 0.700	Acc: 55.00%
Step:   600	Loss: 0.572	Acc: 75.00%
Step:   700	Loss: 0.604	Acc: 67.50%
Step:   800	Loss: 0.611	Acc: 65.00%
Step:   900	Loss: 0.435	Acc: 77.50%
Step:  1000	Loss: 0.493	Acc: 81.67%
Step:  1100	Loss: 0.456	Acc: 81.67%
Step:  1200	Loss: 0.389	Acc: 85.83%
Step:  1300	Loss: 0.414	Acc: 81.67%
Step:  1400	Loss: 0.436	Acc: 80.00%
Step:  1500	Loss: 0.413	Acc: 85.00%
Step:  1600	Loss: 0.461	Acc: 76.67%
Step:  1700	Loss: 0.321	Acc: 87.50%
Step:  1800	Loss: 0.388	Acc: 78.33%
1
[ 38793  24677 211748 ... 147014   3974 195939]
227776
Step:     0	Loss: 0.472	Acc: 73.33%
Step:   100	Loss: 0.415	Acc: 82.50%
Step:   200	Loss: 0.404	Acc: 86.67%
Step:   300	Loss: 0.349	Acc: 86.67%
Step:   400	Loss: 0.

In [ ]:
x_TEST, y_TEST, gt_TEST = cnn_TEST.train(augmented_train, augmented_ground_truth, saver_filename='model_10_epochs_62_window_size', number_epochs=10)

227776
-1.0 0.64705884 -0.6089477
0.5
0
[199471  73984  99619 ... 128754 180765 119397]
227776
Step:     0	Loss: 0.693	Acc: 51.67%
Step:   100	Loss: 0.690	Acc: 55.00%
Step:   200	Loss: 0.685	Acc: 60.83%
Step:   300	Loss: 0.689	Acc: 56.67%
Step:   400	Loss: 0.693	Acc: 50.00%
Step:   500	Loss: 0.656	Acc: 59.17%
Step:   600	Loss: 0.691	Acc: 47.50%
Step:   700	Loss: 0.651	Acc: 56.67%
Step:   800	Loss: 0.651	Acc: 66.67%
Step:   900	Loss: 0.634	Acc: 69.17%
Step:  1000	Loss: 0.556	Acc: 75.00%
Step:  1100	Loss: 0.487	Acc: 79.17%
Step:  1200	Loss: 0.499	Acc: 75.83%
Step:  1300	Loss: 0.450	Acc: 80.83%
Step:  1400	Loss: 0.487	Acc: 76.67%
Step:  1500	Loss: 0.369	Acc: 85.00%
Step:  1600	Loss: 0.387	Acc: 86.67%
Step:  1700	Loss: 0.360	Acc: 86.67%
Step:  1800	Loss: 0.379	Acc: 79.17%
1
[177427  92573  17889 ... 204433 182456 192934]
227776
Step:     0	Loss: 0.412	Acc: 82.50%
Step:   100	Loss: 0.397	Acc: 81.67%
Step:   200	Loss: 0.322	Acc: 85.00%
Step:   300	Loss: 0.349	Acc: 84.17%
Step:   400	Loss: 0.

In [18]:
def submission_per_patch(session, graph, images, img_number, window_size, patch_size=16, stride=16):    
    padding = (window_size-patch_size)//2
    X = graph.get_tensor_by_name("X:0")
    y = graph.get_tensor_by_name("y:0")
    p = graph.get_tensor_by_name("p:0")
    #feed_dict ={X:X_train, p: 1}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("out:0")
    
    test_images = process(images, gt=False)
    test_patches = np.array(img_crop(test_images[img_number-1], patch_size, patch_size, stride, padding))
    if img_number==4:
        print(test_patches.min(), test_patches.max())
    
    
    Z = session.run(op_to_restore, feed_dict={X: test_patches, p: 1})
    print(Z.mean())
    prediction = Z.reshape(-1)
    prediction = np.array([sigmoid(i) for i in prediction])
    prediction = (prediction > 0.5) *1
    print(prediction)
    
    patch_size = 16
    nb = 0
    print("Processing " + str(img_number-1))
    for j in range(0, images[img_number-1].shape[1], patch_size):
        for i in range(0, images[img_number-1].shape[0], patch_size):
            label = int(prediction[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))

In [22]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('../models/' + 'model_10_epochs_32_window_size' + '.ckpt.meta')
saver.restore(sess, '../models/' + 'model_10_epochs_32_window_size' + '.ckpt')
graph = tf.get_default_graph()

X = graph.get_tensor_by_name("X:0")
y = graph.get_tensor_by_name("y:0")
p = graph.get_tensor_by_name("p:0")
#feed_dict ={X:X_train, p: 1}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("out:0")

INFO:tensorflow:Restoring parameters from ../models/model_10_epochs_32_window_size.ckpt


In [23]:
train_data_54, train_gt_32 = crop_and_padding(augmented_train, augmented_ground_truth, 16, 16, 54)

In [36]:
fold_train = np.array(train_data_32[:20000])
fold_test = np.array(train_gt_32[:20000])
fold_labels = np.array([(fold_test[i].mean() > 0.25)*1 for i in range(fold_test.shape[0])])


In [108]:
Z = sess.run(op_to_restore, feed_dict={X: train_data_32[:20000], p: 1})


In [74]:
pred_fold = np.array([(sigmoid(Z[i])>0.5)*1 for i in range(fold_labels.shape[0])])

In [75]:
(pred_fold == fold_labels).mean()

0.93105

In [93]:
Z_id = sess.run(op_to_restore, feed_dict={X: fold_train, p: 1})
Z_ud = sess.run(op_to_restore, feed_dict={X: flip_ud, p: 1})
Z_lr = sess.run(op_to_restore, feed_dict={X: flip_lr, p: 1})
print('jaja')
Z_rot90 = sess.run(op_to_restore, feed_dict={X: flip_rot90, p: 1})
Z_rot180 = sess.run(op_to_restore, feed_dict={X: flip_rot180, p: 1})
Z_rot270 = sess.run(op_to_restore, feed_dict={X: flip_rot270, p: 1})

jaja


In [103]:
pred_fold = np.array([sigmoid(Z[i]) for i in range(fold_labels.shape[0])])
pred_fold_ud = np.array([sigmoid(Z_ud[i]) for i in range(fold_labels.shape[0])])
pred_fold_lr = np.array([sigmoid(Z_lr[i]) for i in range(fold_labels.shape[0])])
pred_fold_rot90 = np.array([sigmoid(Z_rot90[i]) for i in range(fold_labels.shape[0])])
pred_fold_rot180 = np.array([sigmoid(Z_rot180[i]) for i in range(fold_labels.shape[0])])
pred_fold_rot270 = np.array([sigmoid(Z_rot270[i]) for i in range(fold_labels.shape[0])])


In [106]:
pred_mean = (pred_fold + pred_fold_ud + pred_fold_lr + pred_fold_rot90 + pred_fold_rot180 + pred_fold_rot270)/6

In [113]:
((pred_mean>0.55)*1 == fold_labels).mean()

0.93295

In [90]:
flip_ud = np.array([np.flipud(fold_train[i]) for i in range(fold_train.shape[0])])
flip_lr = np.array([np.fliplr(fold_train[i]) for i in range(fold_train.shape[0])])
flip_rot90 = np.array([np.rot90(fold_train[i],1) for i in range(fold_train.shape[0])])
flip_rot180 = np.array([np.rot90(fold_train[i],2) for i in range(fold_train.shape[0])])
flip_rot270 = np.array([np.rot90(fold_train[i],3) for i in range(fold_train.shape[0])])

In [147]:
def submission_per_patch(session, graph, images, img_number, window_size, patch_size=16, stride=16):    
    padding = (window_size-patch_size)//2
    X = graph.get_tensor_by_name("X:0")
    y = graph.get_tensor_by_name("y:0")
    p = graph.get_tensor_by_name("p:0")
    #feed_dict ={X:X_train, p: 1}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("out:0")
    
    test_images = process(images, gt=False)
    test_patches = np.array(img_crop(test_images[img_number-1], patch_size, patch_size, stride, padding))
    
    # datatest augmentation
    flip_ud = np.array([np.flipud(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_lr = np.array([np.fliplr(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_rot90 = np.array([np.rot90(test_patches[i],1) for i in range(test_patches.shape[0])])
    flip_rot180 = np.array([np.rot90(test_patches[i],2) for i in range(test_patches.shape[0])])
    flip_rot270 = np.array([np.rot90(test_patches[i],3) for i in range(test_patches.shape[0])])
    
    
    
    if img_number==4:
        print(test_patches.min(), test_patches.max())
    
    
    Z = session.run(op_to_restore, feed_dict={X: test_patches, p: 1})
    
    # datatest augmentation prediction
    Z_ud = session.run(op_to_restore, feed_dict={X: flip_ud, p: 1})
    Z_lr = session.run(op_to_restore, feed_dict={X: flip_lr, p: 1})
    print('jaja')
    Z_rot90 = session.run(op_to_restore, feed_dict={X: flip_rot90, p: 1})
    Z_rot180 = session.run(op_to_restore, feed_dict={X: flip_rot180, p: 1})
    Z_rot270 = session.run(op_to_restore, feed_dict={X: flip_rot270, p: 1})
    
    
    # Prediction
    pred_fold = np.array([sigmoid(Z[i]) for i in range(test_patches.shape[0])])
    pred_fold_ud = np.array([sigmoid(Z_ud[i]) for i in range(test_patches.shape[0])])
    pred_fold_lr = np.array([sigmoid(Z_lr[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot90 = np.array([sigmoid(Z_rot90[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot180 = np.array([sigmoid(Z_rot180[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot270 = np.array([sigmoid(Z_rot270[i]) for i in range(test_patches.shape[0])])
    
    pred_mean = (pred_fold + pred_fold_ud + pred_fold_lr + pred_fold_rot90 + pred_fold_rot180 + pred_fold_rot270)/6
    
    prediction = (pred_mean>0.6)*1
    
    #print(Z.mean())
    #prediction = Z.reshape(-1)
    #prediction = np.array([sigmoid(i) for i in prediction])
    #prediction = (prediction > 0.5) *1
    #print(prediction)
    
    patch_size = 16
    nb = 0
    print("Processing " + str(img_number-1))
    for j in range(0, images[img_number-1].shape[1], patch_size):
        for i in range(0, images[img_number-1].shape[0], patch_size):
            label = int(prediction[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))

In [115]:
def tf_restore_predict(filename_saver, images, window_size, to_submit_filename=None):
    sess=tf.Session()    
    #First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('../models/' + filename_saver + '.ckpt.meta')
    saver.restore(sess, '../models/' + filename_saver + '.ckpt')
    graph = tf.get_default_graph()

    with open(to_submit_filename, 'w') as f:
        f.write('id,prediction\n')
        for nb_test in range(1, TEST_SIZE+1,1):
            print(nb_test)
            f.writelines('{}\n'.format(s) for s in submission_per_patch(sess, graph, images, nb_test, window_size))
    

In [148]:
tf_restore_predict('model_10_epochs_54_window_size', images, 54, 'submission_10_epochs_54_window_size_test_augmentation_threshold60.csv')

INFO:tensorflow:Restoring parameters from ../models/model_10_epochs_54_window_size.ckpt
1
jaja
Processing 0
2
jaja
Processing 1
3
jaja
Processing 2
4
-1.0 1.0
jaja
Processing 3
5
jaja
Processing 4
6
jaja
Processing 5
7
jaja
Processing 6
8
jaja
Processing 7
9
jaja
Processing 8
10
jaja
Processing 9
11
jaja
Processing 10
12
jaja
Processing 11
13
jaja
Processing 12
14
jaja
Processing 13
15
jaja
Processing 14
16
jaja
Processing 15
17
jaja
Processing 16
18
jaja
Processing 17
19
jaja
Processing 18
20
jaja
Processing 19
21
jaja
Processing 20
22
jaja
Processing 21
23
jaja
Processing 22
24
jaja
Processing 23
25
jaja
Processing 24
26
jaja
Processing 25
27
jaja
Processing 26
28
jaja
Processing 27
29
jaja
Processing 28
30
jaja
Processing 29
31
jaja
Processing 30
32
jaja
Processing 31
33
jaja
Processing 32
34
jaja
Processing 33
35
jaja
Processing 34
36
jaja
Processing 35
37
jaja
Processing 36
38
jaja
Processing 37
39
jaja
Processing 38
40
jaja
Processing 39
41
jaja
Processing 40
42
jaja
Processing 4

In [150]:
def submission_per_patch(session, graph, session2, graph2, images, img_number, window_size, window_size2, patch_size=16, stride=16):    
    
    # Full graph
    padding = (window_size-patch_size)//2
    X = graph.get_tensor_by_name("X:0")
    y = graph.get_tensor_by_name("y:0")
    p = graph.get_tensor_by_name("p:0")
    #feed_dict ={X:X_train, p: 1}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("out:0")
    
    test_images = process(images, gt=False)
    test_patches = np.array(img_crop(test_images[img_number-1], patch_size, patch_size, stride, padding))
    
    # datatest augmentation
    flip_ud = np.array([np.flipud(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_lr = np.array([np.fliplr(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_rot90 = np.array([np.rot90(test_patches[i],1) for i in range(test_patches.shape[0])])
    flip_rot180 = np.array([np.rot90(test_patches[i],2) for i in range(test_patches.shape[0])])
    flip_rot270 = np.array([np.rot90(test_patches[i],3) for i in range(test_patches.shape[0])])
    
    
    
    if img_number==4:
        print(test_patches.min(), test_patches.max())
    
    
    Z = session.run(op_to_restore, feed_dict={X: test_patches, p: 1})
    
    # datatest augmentation prediction
    Z_ud = session.run(op_to_restore, feed_dict={X: flip_ud, p: 1})
    Z_lr = session.run(op_to_restore, feed_dict={X: flip_lr, p: 1})
    print('jaja')
    Z_rot90 = session.run(op_to_restore, feed_dict={X: flip_rot90, p: 1})
    Z_rot180 = session.run(op_to_restore, feed_dict={X: flip_rot180, p: 1})
    Z_rot270 = session.run(op_to_restore, feed_dict={X: flip_rot270, p: 1})
    
    
    # Prediction
    pred_fold = np.array([sigmoid(Z[i]) for i in range(test_patches.shape[0])])
    pred_fold_ud = np.array([sigmoid(Z_ud[i]) for i in range(test_patches.shape[0])])
    pred_fold_lr = np.array([sigmoid(Z_lr[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot90 = np.array([sigmoid(Z_rot90[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot180 = np.array([sigmoid(Z_rot180[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot270 = np.array([sigmoid(Z_rot270[i]) for i in range(test_patches.shape[0])])
    
    # Full graph
    padding2 = (window_size2-patch_size)//2
    X = graph2.get_tensor_by_name("X:0")
    y = graph2.get_tensor_by_name("y:0")
    p = graph2.get_tensor_by_name("p:0")
    #feed_dict ={X:X_train, p: 1}

    #Now, access the op that you want to run. 
    op_to_restore = graph2.get_tensor_by_name("out:0")
    
    #test_images = process(images, gt=False)
    test_patches = np.array(img_crop(test_images[img_number-1], patch_size, patch_size, stride, padding2))
    
    # datatest augmentation
    flip_ud2 = np.array([np.flipud(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_lr2 = np.array([np.fliplr(test_patches[i]) for i in range(test_patches.shape[0])])
    flip_rot902 = np.array([np.rot90(test_patches[i],1) for i in range(test_patches.shape[0])])
    flip_rot1802 = np.array([np.rot90(test_patches[i],2) for i in range(test_patches.shape[0])])
    flip_rot2702 = np.array([np.rot90(test_patches[i],3) for i in range(test_patches.shape[0])])
    
    
    
    if img_number==4:
        print(test_patches.min(), test_patches.max())
    
    
    Z2 = session2.run(op_to_restore, feed_dict={X: test_patches, p: 1})
    
    # datatest augmentation prediction
    Z_ud2 = session2.run(op_to_restore, feed_dict={X: flip_ud2, p: 1})
    Z_lr2 = session2.run(op_to_restore, feed_dict={X: flip_lr2, p: 1})
    print('jaja')
    Z_rot902 = session2.run(op_to_restore, feed_dict={X: flip_rot902, p: 1})
    Z_rot1802 = session2.run(op_to_restore, feed_dict={X: flip_rot1802, p: 1})
    Z_rot2702 = session2.run(op_to_restore, feed_dict={X: flip_rot2702, p: 1})
    
    
    # Prediction
    pred_fold2 = np.array([sigmoid(Z2[i]) for i in range(test_patches.shape[0])])
    pred_fold_ud2 = np.array([sigmoid(Z_ud2[i]) for i in range(test_patches.shape[0])])
    pred_fold_lr2 = np.array([sigmoid(Z_lr2[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot902 = np.array([sigmoid(Z_rot902[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot1802 = np.array([sigmoid(Z_rot1802[i]) for i in range(test_patches.shape[0])])
    pred_fold_rot2702 = np.array([sigmoid(Z_rot2702[i]) for i in range(test_patches.shape[0])])
    
    pred_mean = (pred_fold + pred_fold_ud + pred_fold_lr + pred_fold_rot90 + pred_fold_rot180 + pred_fold_rot270 + pred_fold2 + pred_fold_ud2 + pred_fold_lr2 + pred_fold_rot902 + pred_fold_rot1802 + pred_fold_rot2702)/12
    
    prediction = (pred_mean>0.6)*1
    
    #print(Z.mean())
    #prediction = Z.reshape(-1)
    #prediction = np.array([sigmoid(i) for i in prediction])
    #prediction = (prediction > 0.5) *1
    #print(prediction)
    
    patch_size = 16
    nb = 0
    print("Processing " + str(img_number-1))
    for j in range(0, images[img_number-1].shape[1], patch_size):
        for i in range(0, images[img_number-1].shape[0], patch_size):
            label = int(prediction[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))

In [151]:
def tf_restore_predict(filename_saver, images, window_size, window_size2, filename_saver2, to_submit_filename=None):
    sess=tf.Session()    
    #First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('../models/' + filename_saver + '.ckpt.meta')
    saver.restore(sess, '../models/' + filename_saver + '.ckpt')
    graph = tf.get_default_graph()

    
    sess2=tf.Session()    
    #First let's load meta graph and restore weights
    saver2 = tf.train.import_meta_graph('../models/' + filename_saver2 + '.ckpt.meta')
    saver2.restore(sess2, '../models/' + filename_saver2 + '.ckpt')
    graph2 = tf.get_default_graph()
    
    with open(to_submit_filename, 'w') as f:
        f.write('id,prediction\n')
        for nb_test in range(1, TEST_SIZE+1,1):
            print(nb_test)
            f.writelines('{}\n'.format(s) for s in submission_per_patch(sess, graph, sess2, graph2, images, nb_test, window_size, window_size2 ))
    

In [152]:
tf_restore_predict('model_10_epochs_54_window_size', images, 54, 32, 'model_10_epochs_32_window_size' , 'submission_10_epochs_54_32_window_size_test_augmentation_threshold60.csv')

INFO:tensorflow:Restoring parameters from ../models/model_10_epochs_54_window_size.ckpt
INFO:tensorflow:Restoring parameters from ../models/model_10_epochs_32_window_size.ckpt


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [2304,1] rhs shape= [1024,1]
	 [[Node: save/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable, save/RestoreV2)]]

Caused by op 'save/Assign', defined at:
  File "C:\Users\Brice\Anaconda3\envs\myenvironment\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Brice\Anaconda3\envs\myenvironment\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Brice\Anaconda3\envs\myenvironment\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Brice\Anaconda3\e